## Extracting the Features

In this section, the classes are appropriately labelled at first and separated into 3 classes. Then we convert the preprocessed image files to float variables. Then we compute the 14 features in each of the pictures given. After this imortant step we also extract the statistical textures of the given xray image doubles. Then finally a feature pool is used to collect all the analysed features of all the images given in a class and then the images are extracted by fft, flooring and using discrete wavelet transforms in 2D arrays.

In [1]:
# Importing the required packages from the necessary libraries
from utils import *

import os
import pywt
import cv2
import glob
import itertools
import numpy as np
import seaborn as sn
import pandas as pd
import scipy.io as sio
import skimage.io as io
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.svm import SVC
from tensorflow import keras
from skimage import exposure
from scipy.stats import skew
from scipy.stats import kurtosis
from sklearn import preprocessing
from IPython.display import Image
from skimage.color import rgb2gray
from keras.utils import to_categorical
from skimage.feature import greycomatrix
from keras.callbacks import EarlyStopping
from sklearn.metrics import roc_curve, auc
from sklearn.decomposition import KernelPCA
from tensorflow.keras.utils import plot_model
from skimage.transform import  rescale,resize
from sklearn.preprocessing import MinMaxScaler
from skimage.util import img_as_uint,img_as_ubyte
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
# Importing the source image files preprocessed earlier from the stored directory
class_name = 'pneumonia' #'covid' or 'normal' or 'pneumonia'
source_dir='./dataset/original_images_preprocessed/'+class_name
output_file_name=class_name

In [3]:
# Labelling the three available classes appropriately
if output_file_name == 'covid':
    label = 0
elif output_file_name == 'normal':
    label = 1
else:
    label = 2

In [4]:
kl = keras.layers
# =============================================================================
# im2double
# =============================================================================
# Converting image to double format for image processing
def im2double(img):
    """ convert image to double format """
    min_val = np.min(img.ravel())
    max_val = np.max(img.ravel())
    out = (img.astype('float') - min_val) / (max_val - min_val)
    return out
# =============================================================================
# compute_14_features
# =============================================================================
# The fourteen features that are used to properly classify and process images without any errors
# Area, Mean, Std, Skewness, Kurtosis, Energy, Entropy, Maximum, Mean Abs Deviation,
# Median, Minimum, Range, Root Mean Square, Uniformity

def compute_14_features(region):
    """ Compute 14 features """
    temp_array=region.reshape(-1)
    all_pixels=temp_array[temp_array!=0]
#    Area
    Area = np.sum(all_pixels)
#    mean
    density = np.mean(all_pixels)
#   Std
    std_Density = np.std(all_pixels)
#   skewness
    Skewness = skew(all_pixels)
#   kurtosis
    Kurtosis = kurtosis(all_pixels)
#   Energy
    ENERGY =np.sum(np.square(all_pixels))
#   Entropy
    value,counts = np.unique(all_pixels, return_counts=True)
    p = counts / np.sum(counts)
    p =  p[p!=0]
    ENTROPY =-np.sum( p*np.log2(p));
#   Maximum
    MAX = np.max(all_pixels)
#   Mean Absolute Deviation
    sum_deviation= np.sum(np.abs(all_pixels-np.mean(all_pixels)))
    mean_absolute_deviation = sum_deviation/len(all_pixels)
#   Median
    MEDIAN = np.median(all_pixels)
#   Minimum
    MIN = np.min(all_pixels)
#   Range
    RANGE = np.max(all_pixels)-np.min(all_pixels)
#   Root Mean Square
    RMS = np.sqrt(np.mean(np.square(all_pixels))) 
#    Uniformity
    UNIFORMITY = np.sum(np.square(p))

    features = np.array([Area, density, std_Density,
        Skewness, Kurtosis,ENERGY, ENTROPY,
        MAX, mean_absolute_deviation, MEDIAN, MIN, RANGE, RMS, UNIFORMITY])
    return features
# =============================================================================
# GLDM
# =============================================================================
# GLDM calculates the Gray level Difference Matrix Probability Density Functions for the given image.
# This technique is usually used for extracting statistical texture features of a digital mammogram.

def GLDM(img, distance):
    """ GLDM in four directions """
    pro1=np.zeros(img.shape,dtype=np.float32)
    pro2=np.zeros(img.shape,dtype=np.float32)
    pro3=np.zeros(img.shape,dtype=np.float32)
    pro4=np.zeros(img.shape,dtype=np.float32)
    
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):

            if((j+distance)<img.shape[1]):
                pro1[i,j]=np.abs(img[i,j]-img[i,(j+distance)])
            if((i-distance)>0)&((j+distance)<img.shape[1]):
                pro2[i,j]=np.abs(img[i,j]-img[(i-distance),(j+distance)])
            if((i+distance)<img.shape[0]):
                pro3[i,j]=np.abs(img[i,j]-img[(i+distance),j])
            if((i-distance)>0)&((j-distance)>0):
                pro4[i,j]=np.abs(img[i,j]-img[(i-distance),(j-distance)])

    n=256;
    cnt, bin_edges=np.histogram(pro1[pro1!=0], bins=np.arange(n)/(n-1), density=False)
    Out1 = cnt.cumsum()
    cnt, bin_edges=np.histogram(pro2[pro2!=0], bins=np.arange(n)/(n-1), density=False)
    Out2 = cnt.cumsum()
    cnt, bin_edges=np.histogram(pro3[pro3!=0], bins=np.arange(n)/(n-1), density=False)
    Out3 = cnt.cumsum()
    cnt, bin_edges=np.histogram(pro4[pro4!=0], bins=np.arange(n)/(n-1), density=False)
    Out4 = cnt.cumsum()
    return Out1,Out2,Out3,Out4


In [5]:
# List of images
image_list=os.listdir(source_dir)

# Feature pool is used to concatenate the feature vector for all the images from the respective classes.
feature_pool = np.empty([1,252])
for idx,img_name in enumerate(image_list):
    print(idx)
    img=io.imread(os.path.join(source_dir,img_name))
    img_rescaled=(img-np.min(img))/(np.max(img)-np.min(img)) 
    
    texture_features=compute_14_features(img_rescaled) #texture features
    
    fft_map=np.fft.fft2(img_rescaled)
    fft_map = np.fft.fftshift(fft_map)
    fft_map = np.abs(fft_map)
    YC=int(np.floor(fft_map.shape[1]/2)+1)
    fft_map=fft_map[:,YC:int(np.floor(3*YC/2))]
    fft_features=compute_14_features(fft_map) #FFT features
    
    wavelet_coeffs = pywt.dwt2(img_rescaled,'sym4')
    cA1, (cH1, cV1, cD1) = wavelet_coeffs
    wavelet_coeffs = pywt.dwt2(cA1,'sym4')
    cA2, (cH2, cV2, cD2) = wavelet_coeffs #wavelet features
    wavelet_features=np.concatenate((compute_14_features(cA1), compute_14_features(cH1),compute_14_features(cV1),compute_14_features(cD1)
    ,compute_14_features(cA2), compute_14_features(cH2),compute_14_features(cV2),compute_14_features(cD2)), axis=0)
    
    
    gLDM1,gLDM2,gLDM3,gLDM4=GLDM(img_rescaled,10) #GLDM in four directions
    gldm_features=np.concatenate((compute_14_features(gLDM1), compute_14_features(gLDM2),
                                  compute_14_features(gLDM3),compute_14_features(gLDM4)), axis=0)
    
    
    glcms =greycomatrix(img, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4]) #GLCM in four directions
    glcm_features=np.concatenate((compute_14_features(im2double(glcms[:, :, 0, 0])), 
                                  compute_14_features(im2double(glcms[:, :, 0, 1])),
                                  compute_14_features(im2double(im2double(glcms[:, :, 0, 2]))),
                                  compute_14_features(glcms[:, :, 0, 3])), axis = 0)
    
    feature_vector=np.concatenate((texture_features,fft_features,wavelet_features,gldm_features,glcm_features), axis=0).reshape(1,252)#merge to create a feature vector of 252
    feature_pool=np.concatenate((feature_pool,feature_vector), axis=0)


feature_pool=np.delete(feature_pool, 0, 0)

# Add label to the last column   
feature_pool=np.concatenate((feature_pool,label*np.ones(len(feature_pool)).reshape(len(feature_pool),1)), axis=1)
# Save the created feature pool as a mat file
sio.savemat(output_file_name+'.mat', {output_file_name: feature_pool})

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
